# Data Vault Modeling

A comprehensive guide to **Data Vault 2.0** methodology for building scalable, auditable, and flexible enterprise data warehouses.

---

## What is Data Vault?

**Data Vault** is a hybrid modeling approach designed by Dan Linstedt that combines the best aspects of **3rd Normal Form (3NF)** and **Star Schema** methodologies. It's specifically designed for:

- **Enterprise Data Warehouses (EDW)**
- **Big Data environments**
- **Agile data warehouse development**

### Key Principles

| Principle | Description |
|-----------|-------------|
| **Business Key Driven** | Everything centers around business keys, not surrogate keys |
| **Insert-Only** | No updates or deletes - full audit trail maintained |
| **Parallel Loading** | Hubs, Links, and Satellites can be loaded independently |
| **Separation of Concerns** | Structure (Hubs/Links) separated from context (Satellites) |
| **Historization** | All changes tracked with load timestamps |

## Data Vault 2.0 Methodology

**Data Vault 2.0** extends the original methodology with:

### Enhancements Over 1.0

1. **Hash Keys** - Replace sequence-based surrogate keys for distributed computing
2. **Hash Diffs** - Detect changes in satellite records efficiently
3. **Ghost Records** - Handle missing or unknown relationships
4. **Point-in-Time (PIT) Tables** - Optimize query performance
5. **Bridge Tables** - Pre-join frequently accessed relationships

### Architecture Layers

```
┌─────────────────────────────────────────────────────────────┐
│                    INFORMATION MART                         │
│              (Star Schemas, Reports, Analytics)             │
├─────────────────────────────────────────────────────────────┤
│                    BUSINESS VAULT                           │
│        (Business Rules, Derived Data, Aggregations)         │
├─────────────────────────────────────────────────────────────┤
│                      RAW VAULT                              │
│            (Hubs, Links, Satellites - Source Data)          │
├─────────────────────────────────────────────────────────────┤
│                   STAGING AREA                              │
│              (Raw Source Data, Hash Keys)                   │
└─────────────────────────────────────────────────────────────┘
```

## Core Components: Hubs, Links, and Satellites

Data Vault models consist of three primary entity types:

---

### 🔵 Hubs (Core Business Entities)

Hubs represent **unique business keys** - the core business concepts that never change.

| Column | Description |
|--------|-------------|
| `hub_hash_key` | Primary key (hash of business key) |
| `business_key` | Natural key from source system |
| `load_date` | When record was first loaded |
| `record_source` | Source system identifier |

**Examples:** Customer, Product, Order, Employee

---

### 🔗 Links (Relationships)

Links capture **relationships between Hubs** - many-to-many associations.

| Column | Description |
|--------|-------------|
| `link_hash_key` | Primary key (hash of all hub keys) |
| `hub_1_hash_key` | Foreign key to first hub |
| `hub_2_hash_key` | Foreign key to second hub |
| `load_date` | When relationship was first established |
| `record_source` | Source system identifier |

**Examples:** Customer_Order, Order_Product, Employee_Department

---

### 🛰️ Satellites (Descriptive Context)

Satellites store **descriptive attributes** that change over time.

| Column | Description |
|--------|-------------|
| `parent_hash_key` | FK to parent Hub or Link |
| `load_date` | When this version was loaded (part of PK) |
| `load_end_date` | When this version became inactive |
| `hash_diff` | Hash of all attribute columns |
| `attributes...` | Descriptive data columns |
| `record_source` | Source system identifier |

**Examples:** Customer_Details, Product_Pricing, Order_Status

## Hash Keys vs Business Keys

### Business Keys

**Business keys** are natural identifiers from source systems:

- Customer Number: `CUST-12345`
- Product SKU: `SKU-ABC-001`
- Order ID: `ORD-2024-00001`

### Hash Keys

**Hash keys** are computed surrogate keys using cryptographic hashing:

```python
hash_key = MD5(UPPER(TRIM(business_key)))
# or SHA-256 for larger environments
```

### Why Hash Keys?

| Advantage | Description |
|-----------|-------------|
| **Distributed Computing** | No sequence dependency, parallel loading |
| **Integration Ready** | Same business key = same hash across systems |
| **Consistent Length** | Fixed size regardless of business key length |
| **Deterministic** | Reproducible - can recalculate if needed |
| **Join Performance** | Efficient index lookups |

## Raw Vault vs Business Vault

### Raw Vault (System of Record)

The **Raw Vault** contains data exactly as received from source systems:

- ✅ No business logic applied
- ✅ No transformations or cleansing
- ✅ Full audit trail maintained
- ✅ Source system data preserved verbatim
- ✅ Serves as the "single source of truth"

### Business Vault (Derived Data)

The **Business Vault** extends the Raw Vault with business intelligence:

| Component | Purpose |
|-----------|----------|
| **Same-As Links** | Master data management, entity resolution |
| **Computed Satellites** | Derived/calculated attributes |
| **Bridge Tables** | Pre-joined relationships for performance |
| **PIT Tables** | Point-in-time snapshots for temporal queries |
| **Business Rules** | Applied transformations and validations |

### Design Philosophy

```
RAW VAULT                          BUSINESS VAULT
─────────────                      ──────────────
"What happened"      ───────►      "What it means"
Source data                        Business interpretation
Immutable                          Derived/Calculated
No business rules                  Business rules applied
Audit/Compliance                   Analytics/Reporting
```

In [ ]:
# Data Vault Implementation in Python
# Building a sample data vault model using pandas and hashlib

import pandas as pd
import hashlib
from datetime import datetime, timedelta
from typing import List, Optional
import warnings
warnings.filterwarnings('ignore')

print("Data Vault 2.0 - Python Implementation")
print("=" * 45)

In [ ]:
# Hash Key Generation Utilities

def generate_hash_key(business_key: str, algorithm: str = 'md5') -> str:
    """
    Generate a hash key from a business key.
    
    Args:
        business_key: The natural business key to hash
        algorithm: Hash algorithm ('md5' or 'sha256')
    
    Returns:
        Hexadecimal hash string
    """
    # Standardize: uppercase and strip whitespace
    normalized_key = str(business_key).upper().strip()
    
    if algorithm == 'md5':
        return hashlib.md5(normalized_key.encode()).hexdigest()
    elif algorithm == 'sha256':
        return hashlib.sha256(normalized_key.encode()).hexdigest()
    else:
        raise ValueError(f"Unsupported algorithm: {algorithm}")


def generate_composite_hash_key(keys: List[str], delimiter: str = '||') -> str:
    """
    Generate a hash key from multiple business keys (for Links).
    
    Args:
        keys: List of business keys to combine
        delimiter: Separator between keys
    
    Returns:
        Hexadecimal hash string
    """
    composite_key = delimiter.join(str(k).upper().strip() for k in keys)
    return hashlib.md5(composite_key.encode()).hexdigest()


def generate_hash_diff(attributes: dict) -> str:
    """
    Generate a hash diff from satellite attributes.
    Used for change detection.
    
    Args:
        attributes: Dictionary of attribute values
    
    Returns:
        Hexadecimal hash string
    """
    # Sort keys for consistent hashing
    sorted_values = '||'.join(str(attributes[k]).upper().strip() 
                              for k in sorted(attributes.keys()))
    return hashlib.md5(sorted_values.encode()).hexdigest()


# Demonstrate hash key generation
print("Hash Key Examples:")
print("-" * 50)
print(f"Customer 'CUST-001': {generate_hash_key('CUST-001')}")
print(f"Product 'SKU-ABC':   {generate_hash_key('SKU-ABC')}")
print(f"Link Hash (composite): {generate_composite_hash_key(['CUST-001', 'ORD-001'])}")

In [ ]:
# Sample Source Data - Simulating E-commerce Transactions

# Customer source data
source_customers = pd.DataFrame({
    'customer_id': ['CUST-001', 'CUST-002', 'CUST-003'],
    'customer_name': ['Alice Johnson', 'Bob Smith', 'Carol White'],
    'email': ['alice@email.com', 'bob@email.com', 'carol@email.com'],
    'phone': ['555-0101', '555-0102', '555-0103'],
    'city': ['New York', 'Los Angeles', 'Chicago'],
    'source_system': ['CRM', 'CRM', 'CRM'],
    'extract_date': [datetime.now()] * 3
})

# Product source data
source_products = pd.DataFrame({
    'product_id': ['PROD-001', 'PROD-002', 'PROD-003'],
    'product_name': ['Laptop Pro', 'Wireless Mouse', 'USB-C Hub'],
    'category': ['Electronics', 'Accessories', 'Accessories'],
    'price': [1299.99, 49.99, 89.99],
    'supplier': ['TechCorp', 'GadgetWorld', 'TechCorp'],
    'source_system': ['ERP', 'ERP', 'ERP'],
    'extract_date': [datetime.now()] * 3
})

# Order source data (transactions linking customers and products)
source_orders = pd.DataFrame({
    'order_id': ['ORD-001', 'ORD-002', 'ORD-003', 'ORD-004'],
    'customer_id': ['CUST-001', 'CUST-002', 'CUST-001', 'CUST-003'],
    'product_id': ['PROD-001', 'PROD-002', 'PROD-003', 'PROD-001'],
    'quantity': [1, 2, 1, 1],
    'order_date': [
        datetime(2024, 1, 15),
        datetime(2024, 1, 16),
        datetime(2024, 1, 17),
        datetime(2024, 1, 18)
    ],
    'status': ['Shipped', 'Delivered', 'Processing', 'Shipped'],
    'source_system': ['OMS', 'OMS', 'OMS', 'OMS'],
    'extract_date': [datetime.now()] * 4
})

print("📊 Source Data Loaded")
print(f"   Customers: {len(source_customers)} records")
print(f"   Products:  {len(source_products)} records")
print(f"   Orders:    {len(source_orders)} records")

In [ ]:
# Building the Hub Tables

class DataVaultBuilder:
    """
    Utility class to build Data Vault structures from source data.
    """
    
    def __init__(self, load_date: datetime = None):
        self.load_date = load_date or datetime.now()
    
    def build_hub(self, 
                  source_df: pd.DataFrame,
                  business_key_col: str,
                  record_source_col: str,
                  hub_name: str) -> pd.DataFrame:
        """
        Build a Hub table from source data.
        
        Args:
            source_df: Source DataFrame
            business_key_col: Column containing the business key
            record_source_col: Column containing the source system
            hub_name: Name of the hub (for hash key column naming)
        
        Returns:
            Hub DataFrame
        """
        hub_df = source_df[[business_key_col, record_source_col]].drop_duplicates()
        
        # Generate hash keys
        hub_df[f'{hub_name}_hash_key'] = hub_df[business_key_col].apply(generate_hash_key)
        
        # Add metadata columns
        hub_df['load_date'] = self.load_date
        hub_df = hub_df.rename(columns={record_source_col: 'record_source'})
        
        # Reorder columns
        hub_df = hub_df[[
            f'{hub_name}_hash_key',
            business_key_col,
            'load_date',
            'record_source'
        ]]
        
        return hub_df


# Initialize builder
builder = DataVaultBuilder()

# Build Hub tables
hub_customer = builder.build_hub(
    source_customers, 
    'customer_id', 
    'source_system', 
    'customer'
)

hub_product = builder.build_hub(
    source_products, 
    'product_id', 
    'source_system', 
    'product'
)

hub_order = builder.build_hub(
    source_orders, 
    'order_id', 
    'source_system', 
    'order'
)

print("🔵 HUB_CUSTOMER")
display(hub_customer)

print("\n🔵 HUB_PRODUCT")
display(hub_product)

print("\n🔵 HUB_ORDER")
display(hub_order)

In [ ]:
# Building the Link Tables

def build_link(source_df: pd.DataFrame,
               hub_key_cols: List[str],
               hub_names: List[str],
               link_name: str,
               record_source_col: str,
               load_date: datetime = None) -> pd.DataFrame:
    """
    Build a Link table connecting multiple Hubs.
    
    Args:
        source_df: Source DataFrame
        hub_key_cols: List of columns containing business keys
        hub_names: List of hub names (for FK column naming)
        link_name: Name of the link
        record_source_col: Column containing the source system
        load_date: Load timestamp
    
    Returns:
        Link DataFrame
    """
    load_date = load_date or datetime.now()
    
    # Get unique combinations
    link_df = source_df[hub_key_cols + [record_source_col]].drop_duplicates()
    
    # Generate individual hub hash keys
    for col, hub_name in zip(hub_key_cols, hub_names):
        link_df[f'{hub_name}_hash_key'] = link_df[col].apply(generate_hash_key)
    
    # Generate link hash key (composite of all hub keys)
    link_df[f'{link_name}_hash_key'] = link_df.apply(
        lambda row: generate_composite_hash_key([row[col] for col in hub_key_cols]),
        axis=1
    )
    
    # Add metadata
    link_df['load_date'] = load_date
    link_df = link_df.rename(columns={record_source_col: 'record_source'})
    
    # Select and reorder columns
    hash_key_cols = [f'{link_name}_hash_key'] + [f'{hub}_hash_key' for hub in hub_names]
    link_df = link_df[hash_key_cols + ['load_date', 'record_source']]
    
    return link_df


# Build Link: Customer <-> Order
link_customer_order = build_link(
    source_orders,
    hub_key_cols=['customer_id', 'order_id'],
    hub_names=['customer', 'order'],
    link_name='customer_order',
    record_source_col='source_system'
)

# Build Link: Order <-> Product
link_order_product = build_link(
    source_orders,
    hub_key_cols=['order_id', 'product_id'],
    hub_names=['order', 'product'],
    link_name='order_product',
    record_source_col='source_system'
)

print("🔗 LINK_CUSTOMER_ORDER")
display(link_customer_order)

print("\n🔗 LINK_ORDER_PRODUCT")
display(link_order_product)

In [ ]:
# Building the Satellite Tables

def build_satellite(source_df: pd.DataFrame,
                    parent_key_col: str,
                    parent_name: str,
                    attribute_cols: List[str],
                    record_source_col: str,
                    satellite_name: str,
                    load_date: datetime = None) -> pd.DataFrame:
    """
    Build a Satellite table with descriptive attributes.
    
    Args:
        source_df: Source DataFrame
        parent_key_col: Column containing the parent (Hub/Link) business key
        parent_name: Name of the parent entity
        attribute_cols: List of attribute columns to include
        record_source_col: Column containing the source system
        satellite_name: Name of the satellite
        load_date: Load timestamp
    
    Returns:
        Satellite DataFrame
    """
    load_date = load_date or datetime.now()
    
    # Select relevant columns
    sat_df = source_df[[parent_key_col] + attribute_cols + [record_source_col]].copy()
    
    # Generate parent hash key
    sat_df[f'{parent_name}_hash_key'] = sat_df[parent_key_col].apply(generate_hash_key)
    
    # Generate hash diff for change detection
    sat_df['hash_diff'] = sat_df.apply(
        lambda row: generate_hash_diff({col: row[col] for col in attribute_cols}),
        axis=1
    )
    
    # Add metadata
    sat_df['load_date'] = load_date
    sat_df['load_end_date'] = pd.NaT  # NULL for current records
    sat_df = sat_df.rename(columns={record_source_col: 'record_source'})
    
    # Reorder columns
    ordered_cols = (
        [f'{parent_name}_hash_key', 'load_date', 'load_end_date', 'hash_diff'] +
        attribute_cols +
        ['record_source']
    )
    sat_df = sat_df[ordered_cols]
    
    return sat_df


# Build Satellite: Customer Details
sat_customer_details = build_satellite(
    source_customers,
    parent_key_col='customer_id',
    parent_name='customer',
    attribute_cols=['customer_name', 'email', 'phone', 'city'],
    record_source_col='source_system',
    satellite_name='customer_details'
)

# Build Satellite: Product Details
sat_product_details = build_satellite(
    source_products,
    parent_key_col='product_id',
    parent_name='product',
    attribute_cols=['product_name', 'category', 'price', 'supplier'],
    record_source_col='source_system',
    satellite_name='product_details'
)

# Build Satellite: Order Status
sat_order_status = build_satellite(
    source_orders,
    parent_key_col='order_id',
    parent_name='order',
    attribute_cols=['quantity', 'order_date', 'status'],
    record_source_col='source_system',
    satellite_name='order_status'
)

print("🛰️ SAT_CUSTOMER_DETAILS")
display(sat_customer_details)

print("\n🛰️ SAT_PRODUCT_DETAILS")
display(sat_product_details)

print("\n🛰️ SAT_ORDER_STATUS")
display(sat_order_status)

In [ ]:
# Simulating Change Data Capture (CDC) - Handling Updates

def process_satellite_update(existing_sat: pd.DataFrame,
                             new_records: pd.DataFrame,
                             parent_hash_col: str) -> pd.DataFrame:
    """
    Process updates to a satellite table using CDC pattern.
    Only inserts records that have changed (based on hash_diff).
    
    Args:
        existing_sat: Current satellite DataFrame
        new_records: New incoming satellite records
        parent_hash_col: Name of the parent hash key column
    
    Returns:
        Updated satellite DataFrame with proper historization
    """
    # Get current records (where load_end_date is NULL)
    current_records = existing_sat[existing_sat['load_end_date'].isna()].copy()
    
    # Find changed records by comparing hash_diff
    merged = new_records.merge(
        current_records[[parent_hash_col, 'hash_diff']],
        on=parent_hash_col,
        how='left',
        suffixes=('_new', '_current')
    )
    
    # Filter to only changed or new records
    changes = merged[
        (merged['hash_diff_current'].isna()) |  # New records
        (merged['hash_diff_new'] != merged['hash_diff_current'])  # Changed records
    ].copy()
    
    if len(changes) == 0:
        print("No changes detected.")
        return existing_sat
    
    # Close out old records (set load_end_date)
    changed_keys = changes[parent_hash_col].tolist()
    new_load_date = new_records['load_date'].iloc[0]
    
    existing_sat.loc[
        (existing_sat[parent_hash_col].isin(changed_keys)) &
        (existing_sat['load_end_date'].isna()),
        'load_end_date'
    ] = new_load_date
    
    # Prepare new records for insertion
    new_to_insert = new_records[new_records[parent_hash_col].isin(changed_keys)].copy()
    new_to_insert = new_to_insert.rename(columns={'hash_diff': 'hash_diff'})
    
    # Drop the comparison columns if present
    cols_to_keep = [c for c in new_to_insert.columns if not c.endswith('_new')]
    new_to_insert = new_to_insert[cols_to_keep]
    
    # Combine historical and new records
    result = pd.concat([existing_sat, new_to_insert], ignore_index=True)
    
    print(f"✅ Processed {len(changes)} changed records")
    return result


# Simulate an update: Customer CUST-001 moves to a new city
print("📝 Simulating Customer Update...")
print("   CUST-001: Alice Johnson moved from New York to Boston\n")

updated_customer_data = pd.DataFrame({
    'customer_id': ['CUST-001'],
    'customer_name': ['Alice Johnson'],
    'email': ['alice@email.com'],
    'phone': ['555-0101'],
    'city': ['Boston'],  # Changed from New York
    'source_system': ['CRM'],
    'extract_date': [datetime.now() + timedelta(days=30)]
})

# Build new satellite record
new_sat_record = build_satellite(
    updated_customer_data,
    parent_key_col='customer_id',
    parent_name='customer',
    attribute_cols=['customer_name', 'email', 'phone', 'city'],
    record_source_col='source_system',
    satellite_name='customer_details',
    load_date=datetime.now() + timedelta(days=30)
)

# Process the update
sat_customer_details_updated = process_satellite_update(
    sat_customer_details.copy(),
    new_sat_record,
    'customer_hash_key'
)

print("\n🛰️ SAT_CUSTOMER_DETAILS (After Update - showing history)")
display(sat_customer_details_updated.sort_values(['customer_hash_key', 'load_date']))

In [ ]:
# Complete Data Vault Model Summary

print("=" * 60)
print("📦 DATA VAULT MODEL SUMMARY")
print("=" * 60)

# Display the complete model structure
model_summary = pd.DataFrame({
    'Entity Type': ['Hub', 'Hub', 'Hub', 'Link', 'Link', 'Satellite', 'Satellite', 'Satellite'],
    'Table Name': [
        'HUB_CUSTOMER', 'HUB_PRODUCT', 'HUB_ORDER',
        'LINK_CUSTOMER_ORDER', 'LINK_ORDER_PRODUCT',
        'SAT_CUSTOMER_DETAILS', 'SAT_PRODUCT_DETAILS', 'SAT_ORDER_STATUS'
    ],
    'Business Key(s)': [
        'customer_id', 'product_id', 'order_id',
        'customer_id + order_id', 'order_id + product_id',
        '(attached to HUB_CUSTOMER)', '(attached to HUB_PRODUCT)', '(attached to HUB_ORDER)'
    ],
    'Record Count': [
        len(hub_customer), len(hub_product), len(hub_order),
        len(link_customer_order), len(link_order_product),
        len(sat_customer_details_updated), len(sat_product_details), len(sat_order_status)
    ]
})

display(model_summary)

print("\n" + "=" * 60)
print("📊 Entity Relationship Diagram (Conceptual)")
print("=" * 60)
print("""
                    ┌─────────────────────┐
                    │  SAT_CUSTOMER_      │
                    │     DETAILS         │
                    └─────────┬───────────┘
                              │
                    ┌─────────▼───────────┐
                    │    HUB_CUSTOMER     │
                    │   (customer_id)     │
                    └─────────┬───────────┘
                              │
               ┌──────────────▼──────────────┐
               │   LINK_CUSTOMER_ORDER       │
               │ (customer_id + order_id)    │
               └──────────────┬──────────────┘
                              │
                    ┌─────────▼───────────┐
                    │     HUB_ORDER       │◄───┐  SAT_ORDER_STATUS
                    │    (order_id)       │    │
                    └─────────┬───────────┘────┘
                              │
               ┌──────────────▼──────────────┐
               │    LINK_ORDER_PRODUCT       │
               │  (order_id + product_id)    │
               └──────────────┬──────────────┘
                              │
                    ┌─────────▼───────────┐
                    │    HUB_PRODUCT      │◄───┐  SAT_PRODUCT_DETAILS
                    │   (product_id)      │    │
                    └─────────────────────┘────┘
""")

In [ ]:
# Query Example: Joining the Data Vault Model

print("📊 Sample Query: Customer Order History")
print("=" * 60)

# Join the vault tables to create a business view
customer_order_view = (
    hub_customer
    .merge(
        sat_customer_details_updated[sat_customer_details_updated['load_end_date'].isna()],
        on='customer_hash_key'
    )
    .merge(
        link_customer_order,
        on='customer_hash_key'
    )
    .merge(
        hub_order,
        on='order_hash_key'
    )
    .merge(
        sat_order_status,
        on='order_hash_key'
    )
    .merge(
        link_order_product,
        on='order_hash_key'
    )
    .merge(
        hub_product,
        on='product_hash_key'
    )
    .merge(
        sat_product_details,
        on='product_hash_key'
    )
)

# Create a business-friendly view
business_view = customer_order_view[[
    'customer_id', 'customer_name', 'city',
    'order_id', 'order_date', 'status',
    'product_id', 'product_name', 'price', 'quantity'
]].copy()

business_view['order_total'] = business_view['price'] * business_view['quantity']

print("\n🏢 Business View: Customer Order Details\n")
display(business_view)

## 📚 Key Takeaways

### Core Concepts

1. **Data Vault 2.0** is an enterprise-grade modeling methodology designed for scalability, auditability, and agility.

2. **Three Core Components:**
   - **Hubs** → Store unique business keys (entities)
   - **Links** → Capture relationships between hubs
   - **Satellites** → Hold descriptive, time-variant attributes

3. **Hash Keys** enable distributed, parallel loading without sequence dependencies.

4. **Insert-only pattern** preserves complete audit history.

### Architectural Layers

| Layer | Purpose |
|-------|--------|
| **Raw Vault** | Source data preservation, no business rules |
| **Business Vault** | Derived data, business rules applied |
| **Information Mart** | Star schemas for reporting |

### Best Practices

- ✅ Always normalize business keys (UPPER, TRIM)
- ✅ Use hash_diff for efficient change detection
- ✅ Keep Raw Vault free of business logic
- ✅ Use PIT tables and Bridges for query performance
- ✅ Document all business rules in Business Vault

### When to Use Data Vault

| Good Fit | Not Ideal |
|----------|----------|
| Large enterprise with multiple source systems | Small, single-source applications |
| Frequent schema changes expected | Static, well-defined requirements |
| Strong audit/compliance requirements | Real-time OLTP systems |
| Agile/iterative development | Simple reporting needs |
| Big Data / distributed environments | Traditional BI with limited sources |

---

### Further Reading

- 📖 *Building a Scalable Data Warehouse with Data Vault 2.0* - Dan Linstedt
- 📖 *The Data Vault Guru* - Patrick Cuba
- 🔗 [Data Vault Alliance](https://datavaultalliance.com/)
- 🔗 [dbtvault - dbt Data Vault Package](https://dbtvault.readthedocs.io/)